In [1]:
import sys
import math

import numpy as np

# https://github.com/alexhunsley/numpy-vox-io
# https://github.com/gromgull/py-vox-io
from pyvox.models import Vox
from pyvox.writer import VoxWriter
from pyvox.parser import VoxParser

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [4]:
import magicavoxel

volume_list, palette = magicavoxel.read('vox/horse.vox')
frame_count = len(volume_list)

for frame_index, voxels in enumerate(volume_list):    
    (plane_count, row_count, col_count) = voxels.shape
    for col in range(frame_index, col_count, frame_count):
        voxels[0][0][col]
    for row in range(frame_index, row_count, frame_count):
        voxels[0][row][0]
    for plane in range(frame_index, plane_count, frame_count):
        voxels[plane][0][0]


0 14236
size 31 7 24
size 31 7 24
size 31 7 24
size 31 7 24
